In [4]:
import os, keys
os.environ["OPENAI_API_KEY"] = keys.OPEN_AI_KEY
os.environ["DEV"] = "true"
os.environ["HELIUS_API_KEY"] = keys.HELIUS_KEY

In [5]:
import yaml

from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits.openapi import planner

In [6]:
# Run the below only the first time to get the openapi spec for your API
# !wget https://chatgpt.solanalabs.com/openapi.yaml
# !mv openapi.yaml local_solana_openapi.yaml

In [9]:

with open("local_solana_openapi.yaml") as f:
    raw_solana_api_spec = yaml.load(f, Loader=yaml.Loader)
solana_api_spec = reduce_openapi_spec(raw_solana_api_spec)
print(solana_api_spec.endpoints[3])

('POST /api/handlers/getBalance', 'Accepts Solana publicKey address. Returns the amount of lamports that the account has available.', {'description': 'Accepts Solana publicKey address. Returns the amount of lamports that the account has available.', 'responses': {'description': 'Successful Response', 'content': {'application/json': {'schema': {'title': 'GetBalanceResponse', 'type': 'object', 'properties': {'sol': {'title': 'sol', 'type': 'number'}}}}}}})


In [78]:
# the gpt-4 model is limited access so could not try that
llm = ChatOpenAI(temperature=0, model = "gpt-3.5-turbo-0613") 
# llm = ChatOpenAI(model="gpt-4-0613", temperature=0.0)

In [84]:
# for the local host version we do not have any auth so we pass in an empty dict to requests_wrapper
solana_agent = planner.create_openapi_agent(solana_api_spec, requests_wrapper={},llm = llm)

In [87]:
user_query = ("tell me the user balance in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625")
solana_agent.run(user_query)
# got the expected response



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: query = "user balance in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"
Observation: 1. POST /api/handlers/getBalance with the Solana publicKey address "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625" to retrieve the amount of lamports available in the account.
Thought:I have the plan to retrieve the user balance.
Action: api_controller
Action Input: 1. POST /api/handlers/getBalance with the Solana publicKey address "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"

> Entering new AgentExecutor chain...
I need to execute the first API call to get the balance of a Solana publicKey address. I will use the requests_post tool for this.

Action: requests_post
Action Input: 
{
  "url": "http://localhost:3000/api/handlers/getBalance",
  "data": {
    "address": "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"
  },
  "output_instructions": "sol"
}

Observation: 0.042545265
Thought:The balance of the Solana publicKey addr

'The user balance in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 is 0.042545265.'

In [88]:
user_query = ("Could you provide some info for account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625?")
solana_agent.run(user_query)
# the api call is not formatting the input json correctly. In place of address, it is sending the key account_id



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: "account_info" query with account_id = "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"
Observation: 1. POST /api/handlers/getAccountInfo with the account_id "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625" to retrieve information about the account.
Thought:I have the plan to retrieve the account information.
Action: api_controller
Action Input: POST /api/handlers/getAccountInfo with the account_id "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"

> Entering new AgentExecutor chain...
I need to execute the POST /api/handlers/getAccountInfo API call with the account_id "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625". I will use the requests_post tool for this.

Action: requests_post
Action Input: 
{
  "url": "http://localhost:3000/api/handlers/getAccountInfo",
  "data": {
    "account_id": "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"
  },
  "output_instructions": ""
}

Observation: There is an error with the API reques

ValueError: Could not parse LLM output: `I need to reach out to the API provider for further assistance or information to complete the user's query.`

In [1]:
user_query = ("Could you tell me all the assets owned by owner 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 ?")
solana_agent.run(user_query)
# wrong api call, input: data: ownerAddress in place of just address

NameError: name 'solana_agent' is not defined

### Conclusion:

**Pros/Good Finds:**
- Replacing the older models by `gpt-3.5-turbo-0613` helps even if we're not using the function calling openai tool. 
    - This is because of the API training data fed to the `gpt-3.5-turbo-0613` model
- The advantage over ZERO-SHOT-REACT-DESCRIPTION is that this agent is not generating random API endpoints. It is picking the endpoints from the OPENAPI spec well
- It does good on the URL and the path but not well on the format, discussed below. 
- 

**Cons/Issues found:**
- It messes up when it comes to the `data` of the POST request. It does not follow what is clearly specified in the OPENAPI spec.
- Due to this it in the getAccountInfo example above it is sending the data with `account_id` field that is nowhere in the spec.
- But it does form the right data also sometimes as we can see with the getBalance endpoint above. 
- getAssetsByOwner is having issues because of HELIUS API KEY issues with the local host(not directly related to Agent choice)

